In [7]:
import requests
import json
import re
import pandas as pd

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [3]:
API_KEY = 'RGAPI-5b1aae35-12e5-4c9c-bc40-64b7acdd558b'
FEATURED_GAMES_URL = f'https://na1.api.riotgames.com/lol/spectator/v4/featured-games?api_key={API_KEY}'
SUMMONER_INFO_URL = 'https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'
MATCH_ID_URL = 'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/'
MATCH_INFO_URL = 'https://americas.api.riotgames.com/lol/match/v5/matches/'

In [91]:
# getting a list of currently featured games
games_req = requests.get(FEATURED_GAMES_URL).text
all_games = json.loads(games_req)

# gathering names of all players in each featured game
summoners = []
for game in all_games['gameList']:
    for particpant in game['participants']:
        summoners.append(particpant['summonerName'])

# gathering the encrypted puuids of each summoner and then finding the match_ids of their first 20 recent matches.
match_ids = []
for summoner in summoners:
    url = f'{SUMMONER_INFO_URL}{summoner}?api_key={API_KEY}'
    summoner_req = requests.get(url).text
    summoner_info = json.loads(summoner_req)

    COUNT = 20
    url = f'{MATCH_ID_URL}{summoner_info["puuid"]}/ids?start=0&count={COUNT}&api_key={API_KEY}'
    match_ids_req = requests.get(url).text
    matches = json.loads(match_ids_req)

    match_ids += matches


In [85]:
matches = []

with open('saved-match-ids.txt') as file:
    text = file.readlines()
    pattern = r"NA1_\d{10}"

    matches = re.findall(pattern, text[0])

with open('saved-match-data.txt') as file:
    text = file.readlines()
    pattern = r"NA1_\d{10}"

    matches += re.findall(pattern, text[0]) 

matches = list(set(matches))

In [87]:
kda = []
win = []
vision = []
participant_match_id = []
game_mode = []

for match in matches:
    url = f'{MATCH_INFO_URL}{match}?api_key={API_KEY}'
    match_info_req = requests.get(url)

    if hasattr(match_info_req, 'text') and 'info' in match_info_req.text:
        info = json.loads(match_info_req.text)['info']

        for particpant in info['participants']:
            if 'challenges' in particpant:
                participant_match_id.append(match)
                kda.append(particpant['challenges']['kda'])
                game_mode.append(info['gameMode'])
                win.append(particpant['win'])
                vision.append(particpant['challenges']['visionScorePerMinute'])




In [90]:
df = pd.DataFrame()

df['match_id'] = participant_match_id
df['kda'] = kda
df['vision'] = vision
df['win'] = win
df['game_mode'] = game_mode

df

,match_id,kda,vision,win,game_mode
0,NA1_4299577764,4.200000,0.0,True,ARAM
1,NA1_4299577764,5.200000,0.0,True,ARAM
2,NA1_4299577764,7.666667,0.0,True,ARAM
3,NA1_4299577764,2.166667,0.0,True,ARAM
4,NA1_4299577764,4.500000,0.0,True,ARAM
...,...,...,...,...,...
6225,NA1_4301400131,1.666667,0.0,False,ARAM
6226,NA1_4301400131,2.250000,0.0,False,ARAM
6227,NA1_4301400131,1.166667,0.0,False,ARAM
6228,NA1_4301400131,1.700000,0.0,False,ARAM


In [1]:
df

NameError: name 'df' is not defined